In [1]:
# Import thư viện cần thiết
from mlx_lm import load, generate
from mlx_lm.tuner import train, linear_to_lora_layers
import yaml

/Users/holetinnghia/PycharmProjects/anh-nghai-bot/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Cấu hình Model và Training
# Mày có thể đổi model khác như "Qwen/Qwen1.5-7B-Chat" hoặc "mistralai/Mistral-7B-v0.1"
# Lưu ý: M2 Pro 16GB RAM nên chạy model tầm < 7B tham số hoặc dùng bản 4-bit
model_name = "mistralai/Mistral-7B-v0.1"

# Config cho LoRA (giúp train nhẹ máy)
lora_config = {
    "lora_layers": 16,
    "lora_parameters": {
        "rank": 8,
        "scale": 20.0,
        "dropout": 0.0,
    },
    "iterations": 600, # Số bước train (tăng lên nếu data nhiều)
    "batch_size": 2,   # Giảm xuống 1 hoặc 2 nếu bị tràn RAM
    "learning_rate": 1e-4,
    "max_seq_length": 2048,
}

print(f"--- Đang tải model {model_name} về máy (lần đầu sẽ hơi lâu)... ---")

--- Đang tải model mistralai/Mistral-7B-v0.1 về máy (lần đầu sẽ hơi lâu)... ---


In [3]:
# 2. Load Model & Tokenizer
# Load model gốc
model, tokenizer = load(model_name)

Fetching 10 files: 100%|██████████| 10/10 [06:59<00:00, 42.00s/it]


In [4]:
# 3. Chuẩn bị dữ liệu (QUAN TRỌNG)
# MLX thích dữ liệu dạng jsonl: {"text": "Câu hỏi? \n Câu trả lời."}
# Giả sử mày có file 'data_train.jsonl' và 'data_valid.jsonl'
# Nếu mày chưa có file này, hãy bảo tao, tao viết thêm đoạn code convert data cho.

print("--- Bắt đầu training... ---")

--- Bắt đầu training... ---


In [ ]:
# 4. Bắt đầu train
!mlx_lm.lora --model {model_name} --train --data . --config lora_config.yaml